# ДЗ Парсер сайта

http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/

#### Библиотеки

In [265]:
#!pip3 install requests bs4 pandas
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

#### Получение и преобразование html страницы списка квартир и страницы самой квартиры

In [53]:
table_content = requests.get('http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/')
table_content

<Response [200]>

In [54]:
table_content = BeautifulSoup(table_content.text, 'lxml')
table_content # разобрали страницу на понятный, более менее упорядоченный html код

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="780e99506e03edb5" name="yandex-verification"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Квартиры в Магнитогорске - 3457 объявлений о продаже в Магнитогорске.</title>
<meta content="Квартиры" http-equiv="keywords"/>
<meta content="Объявления от агентств недвижимости и частных лиц о продаже квартир в Магнитогорске. Купите квартиру на рынке вторичного жилья недорого и без посредников " http-equiv="description"/>
<link href="/img/favicon2.png" rel="SHORTCUT ICON"/>
<link href="http://r.citystar.ru/img/_min/realty.adv-list.min.css" rel="stylesheet" type="text/css"/>
<script src="http://r.citystar.ru/js/jquery-1.7.2.min.js?abcd2ba13348f178b17141b445bc99f1917d47af" type="text/javascript"></script>
<script src="http://r.citystar.ru/js/05-jquery-ui-1.8

In [55]:
flat_page = requests.get('http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23240323.html')
flat_page = BeautifulSoup(flat_page.text, 'lxml')
flat_page #посмотрим на код страницы квартиры, чтобы понять, как нам извлекать данные

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="780e99506e03edb5" name="yandex-verification"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Квартира 44,50 м2 по улице Суворова 138 в Магнитогорске - продается за 3550000 рублей. Объявление от продавца  на CityStar.</title>
<meta content="Квартира" http-equiv="keywords"/>
<meta content="Продается Двухкомнатная квартира по адресу: Магнитогорск, ул. Суворова, 138,  район Правобережный за 3550000 рублей." http-equiv="description"/>
<link href="/img/favicon2.png" rel="SHORTCUT ICON"/>
<link href="http://r.citystar.ru/img/_min/realty.adv-card.min.css" rel="stylesheet" type="text/css"/>
<script src="http://r.citystar.ru/js/jquery-1.7.2.min.js?abcd2ba13348f178b17141b445bc99f1917d47af" type="text/javascript"></script>
<script src="http://r.citystar.ru/js/

### Извлечем параметры квартиры с каждой из страниц

Перед извлечением параметров было проведено ручное исследование страниц квартир и страниц со спискоми квартир. Было выявлено, что информация на странице квартиры наиболее полная, а также хранится в более удобном формате. Следовательно парсить данные по квартирам со страницы со списком квартир имеет смысл только для получения айди объявления. Этот айди затем будет использоваться в качестве индекса в нашей таблице с данными.
<img src="../datasets/flat_page.png" width="500"> <img src="../datasets/table_content.png" width="550">

#### 1) Парсим идентификаторы квартир

In [281]:
# Для начала найдем сколько страниц нам придется парсить
div = table_content.find('div', 'pager__pages')
div

<div class="pager__pages"><span class="pager__page_state_current">1</span><a class="pager__page" href="/realty/prodazha-kvartir/?p=2">2</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=3">3</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=4">4</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=5">5</a><a class="pager__page common-link-visited" href="/realty/prodazha-kvartir/?p=6">...</a><a class="pager__page" href="/realty/prodazha-kvartir/?p=21">21</a></div>

In [282]:
numeric_list = [int(i.get_text()) for i in div.find_all('a') if i.get_text().isdigit()]
print(numeric_list)
page_count = max(numeric_list) # Лучше было сделать так: page_count = int(list(div)[-1].get_text())
page_count # Значение можно было получить взяв последний элемент списка, но мало ли как расположит автор сайта нумерацию страниц :)

[2, 3, 4, 5, 21]


21

In [283]:
list_pages = range(1, page_count + 1) # Прямо во время выполнения задания страниц стало 21, вместо 20)
list_pages

range(1, 22)

При просмотре кода страницы с объявлениями видно, что айди квартиры находится в первом и во втором столбцах таблицы. Мы будем брать данные из первого столбца.
<img src="../datasets/flat_id.png">

In [284]:
# Находим на странице таблицу, отсеиваем ненужные строки, получаем список из айдишников квартир
table = table_content.find('table', 'table')
table

<table class="table" id="tbl_10043">
<tr class="headings">
<th class="notebook-column" rowspan="2">
<div class="popup-panel">
<a class="trigger notebook-counter" href="javascript:void(0)" onclick="go('/realty/prodazha-kvartir/?notebook=1')"><span id="note_count_10043">
                            0</span> </a>
<div class="popup" style="width: 140px;">
<div style="padding-bottom: 0px;">
<a class="trigger notebook-counter" href="javascript:void(0)" onclick="go('/realty/prodazha-kvartir/?notebook=1')">Показать отмеченные [+]</a></div>
<a class="cs-cancel-url" href="javascript:void(0)" id="clear_notebook_10043" style="display: none; margin-bottom: 5px;" title="">
                                Очистить блокнот [-]</a>
<div>
<a class="cs-cancel-url" href="/realty/prodazha-kvartir/?" id="cancel_notebook_filter_10043" onclick="go('/realty/prodazha-kvartir/?')" style="display: none;">Показать все [x]</a>
</div>
</div>
</div>
</th>
<th class="heading" rowspan="2">
<div class="heading-block">Да

In [285]:
# Нас инетересует содержание столбеца notebook-column
tr_list = [td for td in table.find_all('tr', recursive=False) if td.find('td', 'notebook-column')]
tr_list[0] # 30 штук

<tr>
<td class="notebook-column"><input name="cs_note_tg" type="checkbox" value="23257068"/></td>
<td class="date">02 окт<br/><span class="time">21:27</span></td><td class="image"><div class="image-block"><span class="ibf" id="ibfbut_23257068" title="Добавить в избранное"></span><a href="/realty/prodazha-kvartir/prodam-kvartiru-23257068.html"><img alt="" src="http://s6.citystar.ru/0q/996d7643c5268e3d58574dbe8f26ab68-crop-87x65.jpg"/></a></div></td><td class="flat-type"><table style="border:none;"><tr><td style="border:none;"><b>2К</b></td><td style="border:none;"></td></tr></table></td><td class="address"><a href="/realty/prodazha-kvartir/prodam-kvartiru-23257068.html">ул Чайковского, 82</a></td><td class="floor">1<span class="slash">/</span>3</td><td class="square">40,3<span class="slash">/</span>30<span class="slash">/</span>5 <span class="colored-text-1">м2</span></td><td class="price"><span>2 600 000</span><span class="colored-text-1"> р.</span></td><td class="price-per-m2">64 516 

In [286]:
id_list = [i.find('input', {'name': 'cs_note_tg'})['value'] for i in tr_list]
link_list = [i.find_all('a')[0]['href'] for i in tr_list]
list(zip(id_list, link_list)) # тоже 30 штук, осталось поставить все "на поток"

[('23257068', '/realty/prodazha-kvartir/prodam-kvartiru-23257068.html'),
 ('23257065', '/realty/prodazha-kvartir/prodam-kvartiru-23257065.html'),
 ('23257064', '/realty/prodazha-kvartir/prodam-kvartiru-23257064.html'),
 ('23257060', '/realty/prodazha-kvartir/prodam-kvartiru-23257060.html'),
 ('23257059', '/realty/prodazha-kvartir/prodam-kvartiru-23257059.html'),
 ('23257058', '/realty/prodazha-kvartir/prodam-kvartiru-23257058.html'),
 ('23257055', '/realty/prodazha-kvartir/prodam-kvartiru-23257055.html'),
 ('23257053', '/realty/prodazha-kvartir/prodam-kvartiru-23257053.html'),
 ('23257052', '/realty/prodazha-kvartir/prodam-kvartiru-23257052.html'),
 ('23257051', '/realty/prodazha-kvartir/prodam-kvartiru-23257051.html'),
 ('23257050', '/realty/prodazha-kvartir/prodam-kvartiru-23257050.html'),
 ('23257049', '/realty/prodazha-kvartir/prodam-kvartiru-23257049.html'),
 ('23257047', '/realty/prodazha-kvartir/prodam-kvartiru-23257047.html'),
 ('23257046', '/realty/prodazha-kvartir/prodam-kvar

In [287]:
#  Вынесем получение списка айдишек в отдельную функцию
SITE = "http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/"

def get_id_list(num_page):
    if num_page == 1:
        page = requests.get(SITE)
    else:
        page = requests.get(SITE + f'?p={num_page}')
    page = BeautifulSoup(page.text, 'lxml')
    tr_list = [td for td in page.find('table', 'table').find_all('tr', recursive=False) if td.find('td', 'notebook-column')]
    id_list = [i.find('input', {'name': 'cs_note_tg'})['value'] for i in tr_list]
    link_list = [i.find_all('a')[0]['href'] for i in tr_list]

    return id_list, link_list

In [288]:
full_id_list, full_link_list = [], []
for num_page in list_pages:
    id_list, link_list = get_id_list(num_page) 
    full_id_list.extend(id_list)
    full_link_list.extend(link_list)

full_id_list, len(full_id_list), full_link_list, len(full_link_list)

(['23257068',
  '23257065',
  '23257064',
  '23257060',
  '23257059',
  '23257058',
  '23257055',
  '23257053',
  '23257052',
  '23257051',
  '23257050',
  '23257049',
  '23257047',
  '23257046',
  '23257044',
  '23257043',
  '23257041',
  '23257040',
  '23257038',
  '23257037',
  '23257036',
  '23257034',
  '23257033',
  '23257032',
  '23257031',
  '23257030',
  '23257029',
  '23257028',
  '23257026',
  '23257025',
  '23257020',
  '23257019',
  '23257018',
  '23257016',
  '23257015',
  '23257014',
  '23257013',
  '23257012',
  '23257011',
  '23257010',
  '23257009',
  '23257008',
  '23257007',
  '23257006',
  '23257005',
  '23257004',
  '23257003',
  '23257002',
  '23257000',
  '23256999',
  '23256998',
  '23256997',
  '23256996',
  '23256993',
  '23256992',
  '23256988',
  '23256986',
  '23256984',
  '23256982',
  '23256981',
  '23256979',
  '23256978',
  '23256977',
  '23256976',
  '23256972',
  '23256967',
  '23256966',
  '23256965',
  '23256964',
  '23256963',
  '23256962',
  '232

In [289]:
len(set(full_id_list)), len(set(full_link_list))

(613, 613)

#### 2) Создаем колонки

Данные будут собираться по 11 параметрам: Ссылка, Цена, Район, Адрес, Этаж, Высота потолка, Планировка, Площадь, Состояние квартиры, Кол-во комнат, Описание.

Из объявлений квартир видно, что данные где-то заполнены не полностью, соответственно в конечной таблице у нас будут пропуски. Так как на каждой странице квартиры таблица с данными разного размера, то ориентироваться будем по названию строк, а затем вставлять их в нужную колонку в нашем датафрейме.

<img src='../datasets/flat_1.png' width=400> <img src='../datasets/flat_2.png' width=400>

In [290]:
# Создаем колонки
columns = ['Ссылка', 'Цена', 'Район', 'Адрес', 'Этаж',
           'Высота потолка', 'Планировка', 'Площадь', 
           'Состояние квартиры', 'Кол-во комнат', 'Описание']
columns

['Ссылка',
 'Цена',
 'Район',
 'Адрес',
 'Этаж',
 'Высота потолка',
 'Планировка',
 'Площадь',
 'Состояние квартиры',
 'Кол-во комнат',
 'Описание']

#### 3) Парсим данные со страницы и через словарь вставляем их в датафрем

Оказалось, что просто идентификатора объявления недостаточно, чтобы перейти на страницу этого объявления...

Так как:
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-23257029.html
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-leninskiy-23257030.html
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-ordzhonikidzevskiy-23182211.html
- http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23188503.html

То есть если квартира в описании имеет название района, то в адресной строке добавляется название этого района, если же район не указан, то окончание строки просто "prodam-kvartiru-айди объявления". Следовательно, чтобы перейти на страницу объявления мы вынуждены вместе с айдишниками парсить район.

Тут я хотел было расстроиться, затем просто вписать вручную транслитерацию районов (их оказывается всего 4), но заметил, что все гораздо проще. Каждая строка таблицы (как же это было очевидно...) имеет внутри себя ссылку на само объявление. Поэтому вместе с айдишками, можно сразу собирать ссылки, которые потом использовать для перехода и сбора данных.

In [291]:
# Возвращаемся на пункт "Парсим идентификаторы квартир"
...
# Теперь все отлично, можно создать датафрейм

In [292]:
df = pd.DataFrame(index=full_id_list, columns=columns)
df['Ссылка'] = full_link_list
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание
23257068,/realty/prodazha-kvartir/prodam-kvartiru-23257...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23257065,/realty/prodazha-kvartir/prodam-kvartiru-23257...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23257064,/realty/prodazha-kvartir/prodam-kvartiru-23257...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23257060,/realty/prodazha-kvartir/prodam-kvartiru-23257...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23257059,/realty/prodazha-kvartir/prodam-kvartiru-23257...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [293]:
# http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23188503.html
flat_page =  requests.get('http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/prodam-kvartiru-pravoberezhnyy-23188503.html')
flat_page = BeautifulSoup(flat_page.text, 'lxml')
flat_page

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="780e99506e03edb5" name="yandex-verification"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Квартира 61,30 м2 по улице Суворова 139 в Магнитогорске - продается за 4050000 рублей. Объявление от продавца  на CityStar.</title>
<meta content="Квартира" http-equiv="keywords"/>
<meta content="Продается Трехкомнатная квартира по адресу: Магнитогорск, ул. Суворова, 139,  район Правобережный за 4050000 рублей." http-equiv="description"/>
<link href="/img/favicon2.png" rel="SHORTCUT ICON"/>
<link href="http://r.citystar.ru/img/_min/realty.adv-card.min.css" rel="stylesheet" type="text/css"/>
<script src="http://r.citystar.ru/js/jquery-1.7.2.min.js?abcd2ba13348f178b17141b445bc99f1917d47af" type="text/javascript"></script>
<script src="http://r.citystar.ru/js/

In [294]:
div = flat_page.find('div', 'adv-main-data')
div

<div class="adv-main-data">
<table>
<tr class="gr-t1">
<td class="group-title" colspan="2">Трехкомнатная квартира по ул. Суворова, 139 в Магнитогорске</td>
</tr>
<tr>
<td class="field-title">Цена</td>
<td class="field"><span class="price">4 050 000</span> р. <span class="colored-text-1">(<strong>66 069</strong> р./м2)</span><br/><a class="common-button_1 card-hypotec-button" href="/mortgageBid/23188503">Подать заявку на ипотеку</a></td>
</tr>
<tr class="even">
<td class="field-title">Район</td>
<td class="field">Правобережный</td>
</tr>
<tr>
<td class="field-title">Адрес</td>
<td class="field">Суворова, 139</td>
</tr>
<tr class="even">
<td class="field-title">Этаж</td>
<td class="field">2<span class="slash">/</span>5</td>
</tr>
<tr>
<td class="field-title">Высота потолка</td>
<td class="field">2,5 м.</td>
</tr>
<tr class="even">
<td class="field-title">Планировка</td>
<td class="field">хрущевка</td>
</tr>
<tr>
<td class="field-title">Площадь</td>
<td class="field"><span class="colored-

In [298]:
tr_list = div.find_all('tr')
tr_list

[<tr class="gr-t1">
 <td class="group-title" colspan="2">Трехкомнатная квартира по ул. Суворова, 139 в Магнитогорске</td>
 </tr>,
 <tr>
 <td class="field-title">Цена</td>
 <td class="field"><span class="price">4 050 000</span> р. <span class="colored-text-1">(<strong>66 069</strong> р./м2)</span><br/><a class="common-button_1 card-hypotec-button" href="/mortgageBid/23188503">Подать заявку на ипотеку</a></td>
 </tr>,
 <tr class="even">
 <td class="field-title">Район</td>
 <td class="field">Правобережный</td>
 </tr>,
 <tr>
 <td class="field-title">Адрес</td>
 <td class="field">Суворова, 139</td>
 </tr>,
 <tr class="even">
 <td class="field-title">Этаж</td>
 <td class="field">2<span class="slash">/</span>5</td>
 </tr>,
 <tr>
 <td class="field-title">Высота потолка</td>
 <td class="field">2,5 м.</td>
 </tr>,
 <tr class="even">
 <td class="field-title">Планировка</td>
 <td class="field">хрущевка</td>
 </tr>,
 <tr>
 <td class="field-title">Площадь</td>
 <td class="field"><span class="colored

In [299]:
# field-title – ключ к словарю, field – его значение, поиск в словаре происходит за O(1), так что все ок
# Подумал сначала сразу получить разделенные данные, но получить их в грязном виде,
# а потом почистить пандасом оказалось проще, чем сразу собрать чистые как надо
# плюс это позволит не заморачиваться над тем, на какую позицию придется ставить найденный элемент

key_list = [key.get_text() for key in div.find_all('td', 'field-title')]
value_list = [value.get_text().replace('\xa0', '') for value in div.find_all('td', 'field')]

data_dict = dict(zip(key_list, value_list))
data_dict['Описание'] = div.find('td', 'note').get_text()
data_dict

# P.S. не понимал почему у одних строк на этой странице класс even, а другие без класса, а потом как дошло...

{'Цена': '4050000р. (66069р./м2)Подать заявку на ипотеку',
 'Район': 'Правобережный',
 'Адрес': 'Суворова, 139',
 'Этаж': '2/5',
 'Высота потолка': '2,5 м.',
 'Планировка': 'хрущевка',
 'Площадь': 'общая 61,3 м2, жилая 46 м2, кухни 6 м2',
 'Состояние квартиры': 'Среднее',
 'Кол-во комнат': 'Трехкомнатная',
 'Этажность дома': '5',
 'Описание': '🏠 Продается уютная 3-комнатная квартира 61,3 м² в отличном состоянии\n📍 Адрес: ул. Суворова, 139, 2 этаж 5-этажного блочного дома\n🔍 О квартире:\n•📐 Площадь: 61.3 м²\n•🏢 Этаж: 2/5\n•🧭 Ориентация окон: Восток-Запад → светлая, весь день солнце\n•🪟 Окна: Пластиковые стеклопакеты\n•🧊 Балкон: Застекленный с выносом\n•⚡ Электрика: Медная проводка\n•💧 Коммуникации: Трубы пластик, установлены водомеры\n•🚻 Санузел: Раздельный в кафеле\n•🧱 Отделка: Полы — стяжка, в коридоре — кафель. Стены ровные. Потолки: натяжной и двухуровневый из гипсокартона.\n🛋️ Остается вся мебель и техника:\n•🍽️ Кухня: кухонный гарнитур, обеденный стол\n•🚪 Прихожая: мебельная стенк

In [300]:
BASE_SITE = 'http://magnitogorsk-citystar.ru'

def get_data(link):
    flat_page = BeautifulSoup(requests.get('http://magnitogorsk-citystar.ru' + link).text, 'lxml')
    div = flat_page.find('div', 'adv-main-data')
    key_list = [key.get_text() for key in div.find_all('td', 'field-title')]
    value_list = [value.get_text().replace('\xa0', '') for value in div.find_all('td', 'field')]
    data_dict = dict(zip(key_list, value_list))
    if div.find('td', 'note'):
        data_dict['Описание'] = div.find('td', 'note').get_text()
    return data_dict

In [301]:
for ind in full_id_list:
    df.loc[str(ind), 'Цена':] = get_data(df.loc[str(ind), 'Ссылка'])

df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание
23257068,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2600000р. (64516р./м2)Подать заявку на ипотеку,NaN,"ул Чайковского, 82",1/3,NaN,NaN,"общая 40,3 м2, жилая 30 м2, кухни 5 м2",NaN,Двухкомнатная,id:45888. \nПродам раздельную двухкомнатную кв...
23257065,/realty/prodazha-kvartir/prodam-kvartiru-23257...,1320000р. (30275р./м2)Подать заявку на ипотеку,NaN,"Степная, 8",1/2,NaN,NaN,"общая 43,6 м2, жилая 25 м2, кухни 9 м2",NaN,Двухкомнатная,id:45839. \n&nbsp; &nbsp; &nbsp;Продам двухком...
23257064,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2400000р. (74766р./м2)Подать заявку на ипотеку,NaN,"Гагарина, 44",1/3,NaN,NaN,"общая 32,1 м2, жилая 17 м2, кухни 6 м2",NaN,Однокомнатная,id:45815. \nЭКСКЛЮЗИВ от АН &laquo;РИО-ЛЮКС&ra...
23257060,/realty/prodazha-kvartir/prodam-kvartiru-23257...,3300000р. (75862р./м2)Подать заявку на ипотеку,NaN,"Гагарина, 67",4/5,NaN,NaN,"общая 43,5 м2, жилая 30 м2, кухни 6 м2",NaN,Двухкомнатная,id:45810. \nВНИМАНИЕ! ЭКСКЛЮЗИВНОЕ ПРЕДЛОЖЕНИЕ...
23257059,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2150000р. (72881р./м2)Подать заявку на ипотеку,NaN,"Ленина пр-т, 98/1",2/9,NaN,NaN,"общая 29,5 м2, жилая 16 м2, кухни 7 м2",NaN,Однокомнатная,id:45805. \n🏘️ Продаётся уютная однокомнатная ...


#### 4) Приводим собранные данные в порядок

In [317]:
flat = df.loc['23188503']
flat

Ссылка                /realty/prodazha-kvartir/prodam-kvartiru-pravo...
Цена                                                            4050000
Район                                                     Правобережный
Адрес                                                     Суворова, 139
Этаж                                                                  2
Высота потолка                                                   2,5 м.
Планировка                                                     хрущевка
Площадь                          общая 61,3 м2, жилая 46 м2, кухни 6 м2
Состояние квартиры                                              Среднее
Кол-во комнат                                             Трехкомнатная
Описание              🏠 Продается уютная 3-комнатная квартира 61,3 м...
Общая площадь                                                      61,3
Жилая площадь                                                        46
Площадь кухни                                                   

In [3]:
df[['Общая площадь', 'Жилая площадь', 'Площадь кухни']] = df['Площадь'].str.split(' ', expand=True).loc[:, [1, 4, 7]]
df['Площадь'].drop()
df.head()

NameError: name 'df' is not defined

In [306]:
df[['Этаж', 'Этажность дома']] = df['Этаж'].str.split('/', expand=True)
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
23257068,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2600000р. (64516р./м2)Подать заявку на ипотеку,NaN,"ул Чайковского, 82",1,NaN,NaN,"общая 40,3 м2, жилая 30 м2, кухни 5 м2",NaN,Двухкомнатная,id:45888. \nПродам раздельную двухкомнатную кв...,"40,3",30,5,3
23257065,/realty/prodazha-kvartir/prodam-kvartiru-23257...,1320000р. (30275р./м2)Подать заявку на ипотеку,NaN,"Степная, 8",1,NaN,NaN,"общая 43,6 м2, жилая 25 м2, кухни 9 м2",NaN,Двухкомнатная,id:45839. \n&nbsp; &nbsp; &nbsp;Продам двухком...,"43,6",25,9,2
23257064,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2400000р. (74766р./м2)Подать заявку на ипотеку,NaN,"Гагарина, 44",1,NaN,NaN,"общая 32,1 м2, жилая 17 м2, кухни 6 м2",NaN,Однокомнатная,id:45815. \nЭКСКЛЮЗИВ от АН &laquo;РИО-ЛЮКС&ra...,"32,1",17,6,3
23257060,/realty/prodazha-kvartir/prodam-kvartiru-23257...,3300000р. (75862р./м2)Подать заявку на ипотеку,NaN,"Гагарина, 67",4,NaN,NaN,"общая 43,5 м2, жилая 30 м2, кухни 6 м2",NaN,Двухкомнатная,id:45810. \nВНИМАНИЕ! ЭКСКЛЮЗИВНОЕ ПРЕДЛОЖЕНИЕ...,"43,5",30,6,5
23257059,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2150000р. (72881р./м2)Подать заявку на ипотеку,NaN,"Ленина пр-т, 98/1",2,NaN,NaN,"общая 29,5 м2, жилая 16 м2, кухни 7 м2",NaN,Однокомнатная,id:45805. \n🏘️ Продаётся уютная однокомнатная ...,"29,5",16,7,9


In [307]:
df['Цена'] = df['Цена'][:].str.split('р.', expand=True).loc[:, 0]
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
23257068,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2600000,NaN,"ул Чайковского, 82",1,NaN,NaN,"общая 40,3 м2, жилая 30 м2, кухни 5 м2",NaN,Двухкомнатная,id:45888. \nПродам раздельную двухкомнатную кв...,"40,3",30,5,3
23257065,/realty/prodazha-kvartir/prodam-kvartiru-23257...,1320000,NaN,"Степная, 8",1,NaN,NaN,"общая 43,6 м2, жилая 25 м2, кухни 9 м2",NaN,Двухкомнатная,id:45839. \n&nbsp; &nbsp; &nbsp;Продам двухком...,"43,6",25,9,2
23257064,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2400000,NaN,"Гагарина, 44",1,NaN,NaN,"общая 32,1 м2, жилая 17 м2, кухни 6 м2",NaN,Однокомнатная,id:45815. \nЭКСКЛЮЗИВ от АН &laquo;РИО-ЛЮКС&ra...,"32,1",17,6,3
23257060,/realty/prodazha-kvartir/prodam-kvartiru-23257...,3300000,NaN,"Гагарина, 67",4,NaN,NaN,"общая 43,5 м2, жилая 30 м2, кухни 6 м2",NaN,Двухкомнатная,id:45810. \nВНИМАНИЕ! ЭКСКЛЮЗИВНОЕ ПРЕДЛОЖЕНИЕ...,"43,5",30,6,5
23257059,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2150000,NaN,"Ленина пр-т, 98/1",2,NaN,NaN,"общая 29,5 м2, жилая 16 м2, кухни 7 м2",NaN,Однокомнатная,id:45805. \n🏘️ Продаётся уютная однокомнатная ...,"29,5",16,7,9


In [316]:
df['Описание'] = df['Описание'].str.replace('^id:\\d+\\.\\s*', '', regex=True)
df.head()

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание,Общая площадь,Жилая площадь,Площадь кухни,Этажность дома
23257068,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2600000,NaN,"ул Чайковского, 82",1,NaN,NaN,"общая 40,3 м2, жилая 30 м2, кухни 5 м2",NaN,Двухкомнатная,Продам раздельную двухкомнатную квартиру!\n\nП...,"40,3",30,5,3
23257065,/realty/prodazha-kvartir/prodam-kvartiru-23257...,1320000,NaN,"Степная, 8",1,NaN,NaN,"общая 43,6 м2, жилая 25 м2, кухни 9 м2",NaN,Двухкомнатная,&nbsp; &nbsp; &nbsp;Продам двухкомнатную разде...,"43,6",25,9,2
23257064,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2400000,NaN,"Гагарина, 44",1,NaN,NaN,"общая 32,1 м2, жилая 17 м2, кухни 6 м2",NaN,Однокомнатная,ЭКСКЛЮЗИВ от АН &laquo;РИО-ЛЮКС&raquo;!\nПрода...,"32,1",17,6,3
23257060,/realty/prodazha-kvartir/prodam-kvartiru-23257...,3300000,NaN,"Гагарина, 67",4,NaN,NaN,"общая 43,5 м2, жилая 30 м2, кухни 6 м2",NaN,Двухкомнатная,ВНИМАНИЕ! ЭКСКЛЮЗИВНОЕ ПРЕДЛОЖЕНИЕ от АН РИО-Л...,"43,5",30,6,5
23257059,/realty/prodazha-kvartir/prodam-kvartiru-23257...,2150000,NaN,"Ленина пр-т, 98/1",2,NaN,NaN,"общая 29,5 м2, жилая 16 м2, кухни 7 м2",NaN,Однокомнатная,🏘️ Продаётся уютная однокомнатная квартира в П...,"29,5",16,7,9


#### 5) Сохраням файл в удобный формат

In [318]:
df.to_csv('../datasets/flat_magnitogorsk.csv')

### Заключение

Сайт делал какой-то извращенец, задание интересное, понравилось, ≈6 часов точно ушло. 
- Парсер первый — айди и ссылки на страницы квартир работает около 2 минут.
- Парсер второй – основные данные по квартирам работает около 5 минут.

Разбираться с типами данных признаков будем уже на анализе данных. Но на последок все же хотел удалить "грязный" столбец Площадь, а ядро уже отключил, поэтому не без помарок в итоговом ноутбуке... 

Доброй ночи 🌛